In [67]:
from tensorflow import keras
model = keras.models.load_model('./model/model_all')

In [77]:
model.summary()
num_of_layers = len(model.layers)

weights = []
bias = []
activation = []
for layer in model.layers:
    if 'conv2d' in layer.get_config()['name']:
        w, b = layer.get_weights()
        print(w.shape)
        print(b.shape)
        continue
    if 'activation' in layer.get_config()['name']:
        continue
    if 'flatten' in layer.get_config()['name']:
        continue
    if 'dropout' in layer.get_config()['name']:
        continue
    w, b = layer.get_weights()
    activation.append(layer.get_config()['activation'])
    weights.append(w)
    bias.append(b)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               30976     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 39,299
Trainable params: 39,299
Non-trainable params: 0
_________________________________________________________________


In [69]:
for w in weights:
    print(w.shape)
for b in bias:
    print(b.shape)

(120, 256)
(256, 32)
(32, 3)
(256,)
(32,)
(3,)


In [70]:
import numpy as np

w = weights
w[0] = w[0].transpose()
w[1] = w[1].transpose()
w[2] = w[2].transpose()

w[0] = np.pad(w[0], ((0, 256 - w[0].shape[0]), (0, 256 - w[0].shape[1])), mode='constant', constant_values=0)
w[1] = np.pad(w[1], ((0, 256 - w[1].shape[0]), (0, 256 - w[1].shape[1])), mode='constant', constant_values=0)
w[2] = np.pad(w[2], ((0, 64 - w[2].shape[0]), (0, 256 - w[2].shape[1])), mode='constant', constant_values=0)

#np.pad(a, ((0,0),(0,3)), mode='constant', constant_values=0)

In [71]:
b = bias

b[0] = np.pad(b[0], (0, 256 - b[0].shape[0]), mode='constant', constant_values=0)
b[1] = np.pad(b[1], (0, 256 - b[1].shape[0]), mode='constant', constant_values=0)
b[2] = np.pad(b[2], (0, 64 - b[2].shape[0]), mode='constant', constant_values=0)

In [72]:
for x in w:
    print(x.shape)
for x in b:
    print(x.shape)

(256, 256)
(256, 256)
(64, 256)
(256,)
(256,)
(64,)


In [73]:
a = np.concatenate((w[0].flatten(), w[1].flatten(), w[2].flatten()), axis=0)
np.savetxt("weights.csv", a, delimiter=",")
a = np.concatenate((b[0].flatten(), b[1].flatten(), b[2].flatten()), axis=0)
np.savetxt("bias.csv", a, delimiter=",")

In [74]:
a = np.genfromtxt('./weights.csv', delimiter=',')
print(a)

[-0.17306007 -0.33779949 -0.08229709 ...  0.          0.
  0.        ]


In [75]:
X = np.array([[0 for x in range(0, 120)], [10 for x in range(0, 120)]])
prediction = model.predict(X)
print(prediction)
for p in prediction:
    print(np.argmax(p, axis=-1))

[[7.0681043e-02 8.2874441e-01 1.0057459e-01]
 [7.9532881e-23 1.0000000e+00 0.0000000e+00]]
1
1


In [78]:

import tensorflow as tf
software_prediction = []    

def dot_product(a, b, n):
    total = []
    for i in range(0, b.shape[1]):
        s = 0
        for j in range(0, n):
            s += a[j] * b[j][i]
        total.append(s)
    return total

def add(a, b, n):
    total = []
    for i in range(0, n):
        total.append(a[i] + b[i])
    return total

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

#minmax = [[float('inf'), float('-inf')] for x in range(0, num_of_layers)]

for line in X:
    in_data = line
    for i in range(0, 3):
#       minmax[i][0] = min(min(in_data), minmax[i][0])
#        minmax[i][1] = max(max(in_data), minmax[i][1])
        pdt = np.matmul(in_data, weights[i], dtype=np.float32)
        in_data = add(pdt, bias[i], len(bias[i]))
        if activation[i] == 'relu':
            in_data = tf.keras.activations.relu(in_data)
        elif activation[i] == 'softmax':
            in_data = in_data
    software_prediction.append(in_data)

software_prediction = np.array(software_prediction)

In [79]:
m = tf.keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)
m.update_state(prediction, software_prediction)
print(m.result().numpy())

1.0


In [80]:
print(software_prediction)

[[ -1.1914414    1.270293    -0.83871895]
 [  9.541855    60.42774    -31.045673  ]]
